#### Data from [Yelp Dataset JSON](https://www.yelp.com/dataset)

In [3]:
# Import dependencies
import pandas as pd
import json

# Import functions
%run -i functions.py


In [ ]:
# Create review dataframe
# Open json data file
data_file = open('../../Data/archive/yelp_academic_dataset_review.json', encoding = 'utf8')
data = []

# Read in data
for line in data_file:
    data.append(json.loads(line))

review_df = pd.DataFrame(data)
data_file.close()

In [4]:
# Create business dataframe
# Open json data file 
data_file = open('../../Data/archive/yelp_academic_dataset_business.json', encoding='utf8')
data = []

# Read in data
for line in data_file:
    data.append(json.loads(line))

business_df = pd.DataFrame(data)
data_file.close()

In [ ]:
# review_df.head()
# business_df.head()

In [ ]:
# review_df.columns
# business_df.columns

#### Transform Review Dataframe

In [ ]:
# Drop reviews from before 1 June 2019 at midnight
review_df = review_df[review_df['date'] >= '2019-06-01 00:00:00']

In [ ]:
# Sort reviews temporally
review_df = review_df.sort_values(by = 'date')

In [ ]:
review_df.head()

In [ ]:
# Drop all positive reviews and reset index
bad_review_df = review_df[review_df['stars'] < 3.0].reset_index()

In [ ]:
# Drop unnecessary columns
bad_review_df.drop(['review_id', 'user_id', 'useful', 'funny', 'cool'], axis = 1, inplace = True)

In [ ]:
bad_review_df.iloc[[10000]]

In [ ]:
# Take only the review coulumn
reviews = bad_review_df['text']


In [ ]:
reviews[12]

#### Transform Business Data

In [5]:
# Drop unnecessary columns
business_df.drop(['stars', 'review_count', 'is_open', 'attributes', 'hours'], axis = 1, inplace = True)

In [6]:
business_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,"Brewpubs, Breweries, Food"


In [18]:
# Split categories column by delimiter and add first column to dataframe
business_df['categories'] = business_df['categories'].str.split(',', expand = True)[0]

In [25]:
# Filter out rows with less than 4 category instances
business_df = business_df.groupby('categories').filter(lambda x : len(x)>3)

In [26]:
business_df['categories'].value_counts()

Restaurants             15290
Food                     6783
Shopping                 5480
Beauty & Spas            4385
Home Services            3793
                        ...  
Bounce House Rentals        4
Personal Assistants         4
Religious Items             4
Shanghainese                4
Oncologist                  4
Name: categories, Length: 919, dtype: int64